name: link_data \
date: 12/12/2024 \
version: 1.0 \
github root: #google colab stream: version 2.0\
author: Justin Sankey, Johanna Ganglbauer \

**description**: Links LCMS data with general sample description, plots and evaluates correlations.

**changes in comparison to previous version**:

**what needs to be implemented**:


In [25]:
# install packages
library('ggplot2')
library('corrplot')

In [26]:
# indicate filepath where general sample description is saved to.
linking_data_file = 'linking_data.csv'
lcms_results_folder = 'example_data_processed'

In [27]:
# read in linking data, LCMS data and put everything together in a data frame
link_data <- data.frame(read.csv(linking_data_file, na.strings=''))

# get all results in lcms result folder and put them together in a dataframe
# curent status: 1 file: todo -> make list!
list_csv_files = list.files(path = lcms_results_folder, pattern = '*.csv')
lcms_data <- read.csv(file.path(lcms_results_folder, list_csv_files[1]), na.strings='', stringsAsFactors=FALSE)
lcms_data <- data.frame(lcms_data)
for (column in c('Below.Detection.Threshold')){
    lcms_data[[column]] = as.logical(lcms_data[[column]])
    lcms_data[is.na(column), column] <- FALSE
}

lcms_data$Calculated.Concentration[lcms_data$Below.Detection.Threshold] <- NA

for (pfas_component in lcms_data$Component.Name){
    link_data[, pfas_component] <- NA
    relevant_concentration = lcms_data[lcms_data$Component.Name == pfas_component,]
    for (row in 1:nrow(link_data)){
        concentration = relevant_concentration$Calculated.Concentration[relevant_concentration$Sample.Name == link_data[row, 'sample_name_in_lcms']]
        link_data[row, pfas_component] = concentration
    }
}

In [ ]:
# evaluate correlations and plot correlation matrix for some components
correlation_matrix = cor(link_data[c('age', 'health_condition', 'PFOA', 'L-PFOS', 'L-PFHxS')], use='complete.obs')

corrplot(correlation_matrix, type = "upper", order = "hclust", na.label = "NA")
#          tl.col = "black", tl.srt = 45,  na.label = "NA")

gg <- ggplot(data=link_data, aes(x=link_data[['L-PFOS']], y=PFOA)) + geom_point() + geom_smooth()
# gg <- gg + geom_smooth(alpha=0.3, method="lm")
print(gg)